This pipeline is used to handle the dataset for this assignment. For this part, since we will not be training the model in this notebook, we should not need a GPU (saving on GPU time)


In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,307 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,056 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [ ]:
#Installs
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qorv7mvh
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-qorv7mvh
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.3 MB/s 
  Created wheel for transformers: filename=transformers-4.26.0.dev0-py3-none-any.whl size=5881991 sha256=55aabb885eae8a604ece1f62d64b116a11b8d54878d4acd339e1d0d95c79ddbe
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahaz00is/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
    

In [ ]:
#Import
from huggingface_hub import notebook_login
from datasets import load_dataset, DatasetDict
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

In [ ]:
#Log in to huggingface
#Get token from huggingface => settings => Acess Tokens 
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
#Loading the dataset
common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="test", use_auth_token=True)

print(common_voice)


Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #4:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #3:   0%|          | 0/1 [00:00<?, ?obj/s]

...





Reading metadata...: 5699it [00:00, 58577.43it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]






Reading metadata...: 1346it [00:00, 50405.20it/s]


Dataset common_voice_11_0 downloaded and prepared to /root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/sv-SE/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 12360
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 5069
    })
})


In [ ]:
#Remove unnecessary parts of the data
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 12360
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5069
    })
})


:## Prepare Feature Extractor, Tokenizer and Data

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Prepare Data

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1aaf2e61fb837413b804938857af61aeb372611d01b80b3f40d9edeb7f354244/common_voice_sv-SE_20466896.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'Du ser ut att ha gjort det här hela livet.'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:

print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/1aaf2e61fb837413b804938857af61aeb372611d01b80b3f40d9edeb7f354244/common_voice_sv-SE_20466896.mp3', 'array': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'Du ser ut att ha gjort det här hela livet.'}


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

#0:   0%|          | 0/6180 [00:00<?, ?ex/s]

#1:   0%|          | 0/6180 [00:00<?, ?ex/s]

#1:   0%|          | 0/2534 [00:00<?, ?ex/s]

#0:   0%|          | 0/2535 [00:00<?, ?ex/s]

In [ ]:
help(common_voice)

Help on DatasetDict in module datasets.dataset_dict object:

class DatasetDict(builtins.dict)
 |  A dictionary (dict of str: datasets.Dataset) with dataset transforms methods (map, filter, etc.)
 |  
 |  Method resolution order:
 |      DatasetDict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, k) -> datasets.arrow_dataset.Dataset
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  align_labels_with_mapping(self, label2id: Dict, label_column: str) -> 'DatasetDict'
 |      Align the dataset's label ID and label name mapping to match an input :obj:`label2id` mapping.
 |      This is useful when you want to ensure that a model's predicted labels are aligned with the dataset.
 |      The alignment in done using the lowercase label names.
 |      
 |      Args:
 |          label2id (:obj:`dict`):
 |              The label name to ID mapping to align the dataset with.
 |          label_column 

In [ ]:
common_voice.save_to_disk("common_voice")

In [ ]:
from google.colab import files
files.download("common_voice")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/file.zip /content/common_voice/

#Zips the content and then you can download it locally.

  adding: content/common_voice/ (stored 0%)
  adding: content/common_voice/train/ (stored 0%)
  adding: content/common_voice/train/dataset_info.json (deflated 51%)
  adding: content/common_voice/train/dataset.arrow (deflated 90%)
  adding: content/common_voice/train/state.json (deflated 41%)
  adding: content/common_voice/dataset_dict.json (stored 0%)
  adding: content/common_voice/test/ (stored 0%)
  adding: content/common_voice/test/dataset_info.json (deflated 51%)
  adding: content/common_voice/test/dataset.arrow (deflated 90%)
  adding: content/common_voice/test/state.json (deflated 41%)


In [ ]:
#THIS CELL SHOULD BE USED TO SAVE THE DATASET IN THE CLOUD